# Preprocessing

In [151]:
import pandas as pd
import numpy as np
import pickle

from data.turk import TurkResults2Label
from data.dao import LabelGetter

In [132]:
batch1 = pd.read_csv('C:/users/tom work/downloads/Batch_2431673_batch_results.csv').drop_duplicates(subset='_id')
batch2 = pd.read_csv('C:/users/tom work/downloads/Batch_2431727_batch_results.csv').drop_duplicates(subset='_id')

In [133]:
batch1.index = batch1._id
batch2.index = batch2._id

Remove workers that answered the same question twice...

In [134]:
answers = batch1[batch1.Worker != batch2.Worker]

In [135]:
answers['Answer2'] = batch2.ix[answers.index].Answer

C:\Users\Tom Work\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [249]:
set(answers.Answer.values)

{'Alcohol Related::Discussion',
 'Alcohol Related::Promotional Content',
 'First Person - Alcohol::Casual Drinking',
 'First Person - Alcohol::Heavy Drinking',
 'First Person - Alcohol::Looking to drink',
 'First Person - Alcohol::Reflecting on drinking',
 'Not Alcohol Related',
 '{}'}

Some of them have no answer {} ??????

In [137]:
labels1= answers.Answer[answers.Answer != '{}'].apply(TurkResults2Label.parse_to_labels)
labels2= answers.Answer2[answers.Answer2 != '{}'].apply(TurkResults2Label.parse_to_labels)

In [138]:
labels1.shape

(947,)

In [139]:
labels2.shape

(973,)

In [140]:
batch1['label1'] = labels1

In [141]:
batch1['label2'] = labels2

In [144]:
labeled = batch1.dropna(subset=['label1', 'label2'])

In [147]:
agreed = labeled[labeled.label1 == labeled.label2]

In [148]:
agreed.shape

(476, 9)

# Kappa Scores

In [149]:
from sklearn.metrics import cohen_kappa_score

Alcohol

In [183]:
L = LabelGetter(labeled)

In [184]:
Xalc1, yalc1 = L._get_labels('alcohol', 'label1')
Xalc2, yalc2 = L._get_labels('alcohol', 'label2')
cohen_kappa_score(yalc1, yalc2)

0.57139053805809192

FPA

In [271]:
X_alc, y_alc = Xalc1[yalc1==yalc2], yalc1[yalc1==yalc2]

In [272]:
X_alc.shape

(760, 9)

In [273]:
L = LabelGetter(alc_agree)
Xfpa1, yfpa1 = L._get_labels('first_person', 'label1')
Xfpa2, yfpa2 = L._get_labels('first_person', 'label2')
cohen_kappa_score(yfpa1, yfpa2)

0.2822212537559925

FPL

In [274]:
X_fpa, y_fpa = Xfpa1[yfpa1==yfpa2], yfpa1[yfpa1==yfpa2]

In [275]:
X_fpa.shape

(330, 9)

In [264]:
L = LabelGetter(fpa_agree)
Xfpl1, yfpl1 = L._get_labels('first_person_level', 'label1')
Xfpl2, yfpl2 = L._get_labels('first_person_level', 'label2')
cohen_kappa_score(yfpl1, yfpl2)


0.35722100656455125

In [276]:
X_fpl, y_fpl = Xfpl1[yfpl1==yfpl2], yfpl1[yfpl1==yfpl2]

In [277]:
X_fpl.shape

(113, 9)

Looks like people are pretty good at deciding of a tweet is alcohol related, but FPA and FPL are more ambiguous.

# Testing Classifiers

In [250]:
clf_alc = pickle.load(open('pickles/clf_alc_UPDATED.p', 'rb'))
clf_fpa = pickle.load(open('pickles/clf_fpa_UPDATED.p', 'rb'))
clf_fpl = pickle.load(open('pickles/clf_fpl_double_labeled', 'rb'))

In [305]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score

In [306]:
metrics = [accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score]

In [307]:
def print_metrics(y_pred, y_true):
    for metric in metrics:
        kwargs = {}
        if metric in [f1_score, precision_score, recall_score]:
            kwargs["average"] = "weighted"
        print(metric.__name__, metric(y_true=y_true, y_pred=y_pred, **kwargs))

In [308]:
print_metrics(clf_alc.predict(X_alc), y_alc)

accuracy_score 0.761842105263
f1_score 0.829727187206
confusion_matrix [[138 109]
 [ 72 441]]
classification_report              precision    recall  f1-score   support

          0       0.66      0.56      0.60       247
          1       0.80      0.86      0.83       513

avg / total       0.75      0.76      0.76       760

precision_score 0.801818181818
recall_score 0.859649122807


C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.

In [309]:
print_metrics(clf_fpa.predict(X_fpa), y_fpa)

accuracy_score 0.669696969697
f1_score 0.764578833693
confusion_matrix [[ 44  98]
 [ 11 177]]
classification_report              precision    recall  f1-score   support

          0       0.80      0.31      0.45       142
          1       0.64      0.94      0.76       188

avg / total       0.71      0.67      0.63       330

precision_score 0.643636363636
recall_score 0.941489361702


C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.

In [310]:
print_metrics(clf_fpl.predict(X_fpl), y_fpl)

accuracy_score 0.610619469027
f1_score 0.628726257946
confusion_matrix [[36 12 14]
 [ 8 26  5]
 [ 3  2  7]]
classification_report              precision    recall  f1-score   support

          0       0.77      0.58      0.66        62
          1       0.65      0.67      0.66        39
          2       0.27      0.58      0.37        12

avg / total       0.67      0.61      0.63       113

precision_score 0.673186999406
recall_score 0.610619469027
